In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.distributions import Normal

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [13]:
Normal(1.0,0.0).log_prob(0.3)

<tf.Tensor 'Normal_7/log_prob/sub:0' shape=() dtype=float32>

In [3]:
batch_size = 100
num_classes = 10
epochs = 100

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

x_train = mnist.train.images
y_train = mnist.train.labels

x_test = mnist.test.images
y_test = mnist.test.labels

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
55000 train samples
10000 test samples


In [4]:
model = Sequential()
model.add(Dense(1000, activation='relu', input_shape=(784,)))
#model.add(Dropout(0.5))
#model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1)
                    #validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                10010     
Total params: 795,010
Trainable params: 795,010
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
55000/55000 [==============================] - 4s 81us/step - loss: 0.2356 - acc: 0.9319
Epoch 2/100
55000/55000 [==============================] - 2s 32us/step - loss: 0.0909 - acc: 0.9723
Epoch 3/100
55000/55000 [==============================] - 2s 32us/step - loss: 0.0571 - acc: 0.9823
Epoch 4/100
55000/55000 [==============================] - 2s 32us/step - loss: 0.0369 - acc: 0.9890
Epoch 5/100
55000/55000 [==============================] - 2s 32us/step - loss: 0.0263 - acc: 0.9925
Epoch 6/100
55000/55000 [==

In [13]:
from keras.layers.core import Lambda
from keras import backend as K
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model
K.exp
def custom_activation(x):
    return (K.sigmoid(x) * 5) - 1

input_ = Input(shape=(10,), name='aux_input')
x = Dense(512, activation='relu')(input_)
x = Lambda(lambda x: K.dropout(x, level=0.5))(x)
x = Dense(512, activation='relu')(x)
x = Lambda(lambda x: K.dropout(x, level=0.5))(x)
mean = Dense(1, activation='linear')(x)
std = Dense(1, activation=lambda y: K.exp(y))(x)
out = keras.layers.concatenate([mean, std])

model = Model(inputs=[input_], outputs=[out])

def customLoss(yTrue,yPred):
    mean = yPred[:,0]
    std = yPred[:,1]
    return -K.mean(Normal(mean, std).log_prob(yTrue))

model.compile(loss=customLoss,
      optimizer=Adam(),
      metrics=['mse'])

X = np.random.randn(1000, 10)
y = np.random.randn(1000, 1)
model.fit(X, y,
            batch_size=100,
            epochs=10,
            verbose=1)

Epoch 1/10
1000/1000 [==============================] - 0s 215us/step - loss: 1.5087 - mean_squared_error: 1.6214
Epoch 2/10
1000/1000 [==============================] - 0s 20us/step - loss: 1.4664 - mean_squared_error: 1.5448
Epoch 3/10
1000/1000 [==============================] - 0s 21us/step - loss: 1.4418 - mean_squared_error: 1.5594
Epoch 4/10
1000/1000 [==============================] - 0s 20us/step - loss: 1.4340 - mean_squared_error: 1.4109
Epoch 5/10
1000/1000 [==============================] - 0s 20us/step - loss: 1.4229 - mean_squared_error: 1.5302
Epoch 6/10
1000/1000 [==============================] - 0s 20us/step - loss: 1.4170 - mean_squared_error: 1.4223
Epoch 7/10
1000/1000 [==============================] - 0s 21us/step - loss: 1.4139 - mean_squared_error: 1.4361
Epoch 8/10
1000/1000 [==============================] - 0s 21us/step - loss: 1.4136 - mean_squared_error: 1.4451
Epoch 9/10
1000/1000 [==============================] - 0s 20us/step - loss: 1.4113 - mean_squa

In [15]:
model.predict(X)[:,0]

array([ 4.47799042e-02,  4.29928787e-02,  4.01972197e-02, -5.85811846e-02,
        2.10425188e-03, -2.60138493e-02,  5.67804649e-02,  1.00457063e-02,
       -2.87980810e-02,  5.70625886e-02,  7.02490844e-03,  6.30685017e-02,
        1.74817115e-01,  1.33323614e-02,  6.31371215e-02, -8.13022163e-03,
       -6.16925117e-03, -3.22691374e-03,  5.32552674e-02, -3.23087759e-02,
        1.10041142e-01,  6.35125637e-02,  6.70273677e-02,  9.77277979e-02,
        7.79308379e-02,  3.39864492e-02,  3.32650281e-02,  9.96438637e-02,
        4.46981750e-03, -7.69223087e-03,  1.74940433e-02,  1.12803606e-02,
        1.99649140e-01,  8.48856568e-02,  1.09065706e-02,  2.27597984e-03,
        5.04011959e-02,  1.17061108e-01, -1.60697252e-02,  1.52494416e-01,
        4.56416868e-02, -1.84933487e-02,  2.44099461e-02,  8.49705040e-02,
        6.35574758e-02,  2.17686221e-02,  8.85499716e-02,  2.78269444e-02,
        4.27050143e-02,  6.08682111e-02,  8.26383084e-02,  4.96780649e-02,
       -9.06294212e-03,  

In [5]:
import numpy as np
np.random.randn(100, 10)

array([[ 9.88268351e-01, -2.66562367e-01, -4.06048092e-01,
         3.62282080e-01,  2.33216644e+00,  2.74114247e-01,
        -1.26792218e-01, -1.57952467e-01, -2.56387103e-01,
        -1.42417004e+00],
       [ 4.63833056e-01,  8.19325638e-01,  6.23811408e-01,
        -9.05841836e-01, -2.08988960e-01,  1.35932503e-01,
        -5.30648074e-01, -1.14421272e+00,  1.04734635e+00,
         2.45789517e+00],
       [ 3.94053137e-01, -4.12011009e-01, -6.79682617e-02,
        -5.45975787e-01, -3.62446675e-01, -5.38446720e-01,
         3.21661721e-01, -2.06493467e-01,  8.95739566e-01,
         4.08495424e-01],
       [ 1.76119885e+00,  1.45409917e+00, -2.75408230e-01,
        -1.68621585e+00, -9.35414356e-01,  5.10298756e-01,
        -1.53253713e+00,  7.92813440e-01, -4.88278738e-01,
         9.38601050e-01],
       [-9.86655591e-01,  2.17671246e+00,  6.30484760e-01,
         2.16200532e-01,  9.48792569e-01,  1.78784137e+00,
        -6.18274406e-01, -2.60141342e+00,  5.34394413e-01,
         1.

In [33]:
X.shape[-1]

10